# Phase Kickback

Phase kickback [[1](#childs),[2](#LinLin),[3](#Wolf)] is an important and highly used primitive in quantum computing. The algorithm deals with "kicking" the result of a function to the phase of a quantum state so it can be wisely manipulated with interferences to achieve the desired result. Every quantum algorithm can be decomposed into these steps: 1) Encoding the data, 2) Manipulating the data, and 3) Extracting the result. The phase kickback primitive is a key step in manipulating the data, enabling the extraction of the result. See the [Deutsch-Jozsa](https://github.com/Classiq/classiq-library/blob/main/algorithms/deutsch_jozsa/deutsch_jozsa.ipynb) and [Simon's](https://github.com/Classiq/classiq-library/blob/main/algorithms/simon/simon.ipynb) algorithms for concrete examples.

The standard way [[4](#NielsenChuang)] of applying a classical function $f:\{0,1\}^n \rightarrow \{0,1\}$ on quantum states is by utilizing the oracle model 
$$
O_f (|x \rangle_n|y \rangle_1) = |x \rangle_n|y \oplus f(x) \rangle_1
$$
where $|x \rangle$ and $|y \rangle$ are the argument and target quantum states, respectively, and $\oplus$ is the addition modulo $2$ or the XOR operation. 

The phase kickback primitive takes the oracle $O_f$ as input, and performs the operation
$\begin{equation}
|x \rangle\rightarrow (-1)^{f(x)}|x \rangle
\end{equation}$. 
This can be applied to a superposition of states 
$\begin{equation}
\sum_{i=0}^{2^n-1}\alpha_i|x_i \rangle\rightarrow (-1)^{f(x_i)}\alpha_i|x_i \rangle
\end{equation}$
as well, where $\alpha_i$ are the amplitudes of the $|x_i \rangle$ state. 

The operation is achieved by simply applying the oracle $O_f$ to the state target quantum state $|- \rangle=\frac{1}{\sqrt{2}}|0 \rangle-\frac{1}{\sqrt{2}}|1 \rangle$:
$$
O_f|x \rangle |- \rangle = (-1)^{f(x)}|x \rangle |- \rangle
$$
and then arriving at the desired transformation $|x \rangle\rightarrow (-1)^{f(x)}|x \rangle$. 

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/PK_high_level.png" alt="Phase Kickback High Level" />
</div>

See full details in the [mathematical description](#mathematical-description).

**Classiq Concepts**
* [within-apply](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/within-apply/ )
* [in-place XOR](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/numeric-assignment/ )

**Related Algorithms**
* [Deutsch-Jozsa algorithm](https://github.com/Classiq/classiq-library/blob/main/algorithms/deutsch_jozsa/deutsch_jozsa.ipynb)
* [Simon's algorithm](https://github.com/Classiq/classiq-library/blob/main/algorithms/simon/simon.ipynb)

<details markdown>
<summary markdown> NOTE </summary>

Another trick is phase kickback ([video 1](https://www.youtube.com/watch?v=iLcQ-X6QzvU), [video 2](https://www.youtube.com/watch?v=455pmYaZXKw)). A unitary $U$ is applied to an eigensystem $U| phi \rangle=e^{i\phi}|phi\rangle$ which is controlled by a qubit in the $| + \rangle=\frac{1}{\sqrt{2}}|0\rangle+\frac{1}{\sqrt{2}}|1\rangle$, and the phase of the eigenvalue of $U$ is kicked back to a relative phase in the controlled qubit:
$\begin{equation}
CU| + \rangle| \phi \rangle=\frac{1}{\sqrt2}(|0\rangle| \phi \rangle + e^{i\phi}|1\rangle| \phi \rangle) = \frac{1}{\sqrt{2}}(|0\rangle+e^{i\phi}|1\rangle)| \phi \rangle
\end{equation}$

This useful trick is not the same as the Phase Kickback described in the literature [[1](#childs),[2](#LinLin),[3](#Wolf)]
</details>

## Guided Implementation

When using the Classiq platform (as for any development platform), it is recommended to design your code at the functional level in terms of functions. The above figure indicates which functional building blocks you need, so you can now implement them, one function at a time, and then connect them.

Start with the **Prepare** $| - \rangle$ building block using the `prepare_minus` function. It is implemented by sequentially applying the `X` (NOT) gate that transforms $|0\rangle$ into $X|0\rangle=|1\rangle$, and then applying the Hadamard `H` gate that takes the $|1\rangle$ state into the desired $H|1\rangle=| - \rangle$ state:

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/prepare_minus.gif" alt="Prepare Minus Function" />
</div>

In [1]:
from classiq import *


@qfunc
def prepare_minus(target: Output[QBit]):
    allocate(out=target, num_qubits=1)
    X(target)
    H(target)

Now, define the oracle function that implements $O_f| x \rangle| target \rangle=| x \rangle |target \oplus f(x)\rangle$ for the function $f(x)= (x==0)$, such that the output is $1$ only if the input $x$ equals $0$.
$\begin{equation}
O_f(| x \rangle| target \rangle) = | x \rangle |target \oplus (x==0)\rangle
\end{equation}$

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/oracle_function_new.gif" alt="Oracle Function" />
</div>

With Classiq, the XOR operation ($\oplus$) is implemented intuitively with `^=` :

In [2]:
@qfunc
def oracle_function(target: QBit, x: QNum):
    target ^= x == 0

Combine the two building blocks for performing the phase kickback. You are not interested in the $| - \rangle$ state as its only purpose is to implement $| x \rangle\rightarrow (-1)^{f(x)}| x \rangle$. For such cases (that are common in quantum computing), the Qmod language offers the `within-apply` ([read more](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/within-apply/ ) ) statement. Apply everything intended for use by another specific function in the `within`, and use it for the specific thing in the `apply`

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/Compute_uncompute.gif" alt="compute and uncompute" />
</div>

In [3]:
@qfunc
def oracle_phase_kickback(x: QNum):
    target = QBit("target")
    within_apply(
        within=lambda: prepare_minus(target), apply=lambda: oracle_function(target, x)
    )

<details>

<summary>Note! </summary>

This is how the native Qmod syntax in the IDE looks:

```
qfunc oracle_phase_kickback(x:qnum){
  target:qbit;
  within{prepare_minus(target);} apply{oracle_function(x,target);}
}
```
which is more intuitive.
</details>

Now you can use the `oracle_phase_kickback` function. To utilize the power of quantum parallelism, apply the `oracle_phase_kickback` on a superposition of numbers. Creating a superposition is commonly done with the Hadamard transform in quantum computing, which means applying the Hadamard gate `H` on all the qubits. Its application on the $|0\rangle$ state:
$\begin{equation}
H^{\otimes n}|0\rangle_n = \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1}| i \rangle_n
\end{equation}$

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/Hadamard_PK_new.gif" alt="Hadamard Transform" />
</div>


Therefore, the phase kickback primitive is constructed by the following steps:

1. Initializing the `qnum` type with $4$ qubits using the `allocate` function such that $| x \rangle=|0\rangle$.
2. Applying the Hadamard transform using the `hadamard_transform` function such that $| x \rangle = \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1}| i \rangle_n$.
3. Applying the phase kickback primitive with the `oracle_phase_kickback` function such that $| x \rangle = \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1}(-1)^{i==0}| i \rangle_n$.

In [4]:
@qfunc
def main(x: Output[QNum]):
    allocate(num_qubits=4, out=x)
    hadamard_transform(x)
    oracle_phase_kickback(x)


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/2uG1tsBMFj1MHSPifo9RQW60wUB?version=0.70.0


<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/PK_algorithm_overview.gif" alt="Phase Kickback overview" />
</div>

Verify that the quantum program actually operates as expected; receiving a minus $(-)$ phase only for the state $|0\rangle$, that is:
$\begin{equation}
| x \rangle = \frac{1}{\sqrt{2^4}}(|1\rangle+| 2 \rangle+\dots +| 15 \rangle-|0\rangle)
\end{equation}$
To check, use the built-in Classiq state vector simulator from the IDE:

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/PK_execution_new.gif" alt="Execution with State Vector" />
</div>

Indeed, we receive the minus $(-)$ phase only for the zero state, as expected.
The result shows bit strings of five bits rather than just four. This includes the `target` qubit measurement that always results in 0.

To see how this phase kickback is actually implemented in practice see the [Deutsch-Josza example](https://github.com/Classiq/classiq-library/blob/main/algorithms/deutsch_jozsa/deutsch_jozsa.ipynb) and [Simon's example](https://github.com/Classiq/classiq-library/blob/main/algorithms/simon/simon.ipynb).

## Mathematical Description

The starting point is the definition 
$\begin{equation}
O_f (| x \rangle_n| y \rangle_1) = | x \rangle_n|y \oplus f(x)\rangle_1
\end{equation}$
for $f:\{0,1\}^n \rightarrow \{0,1\}$.

Applying $O_f$ on $| x \rangle| - \rangle$ results in this:
$\begin{equation}
O_f (| x \rangle| - \rangle) = \frac{1}{\sqrt{2}}(| x \rangle |0 \oplus f(x)\rangle - | x \rangle |1 \oplus f(x)\rangle)
\end{equation}$

The expression $0 \oplus f(x)$ equals $f(x)$ because $0\oplus0=0$ and $0\oplus1=1$.

The expression $1\oplus f(x)$ equals $\overline{f(x)}$; i.e., `NOT` $f(x)$ since $1\oplus1=0$ and $1\oplus0=1$.

Therefore:

$\begin{equation}
O_f (| x \rangle| - \rangle) = | x \rangle\frac{1}{\sqrt{2}}(|f(x)\rangle - |\overline{f(x)}\rangle).
\end{equation}$

If $f(x)=0$ the target state is $\frac{1}{\sqrt{2}}(|0\rangle - |1\rangle) = | - \rangle = (-1)^{f(x)}| - \rangle$.

If $f(x)=1$ the target state is $\frac{1}{\sqrt{2}}(|1\rangle - |0\rangle) = -| - \rangle = (-1)^{f(x)}| - \rangle$.

Therefore:

$\begin{equation}
O_f (| x \rangle| - \rangle) = (-1)^{f(x)}| x \rangle| - \rangle.
\end{equation}$



## All the Code Together

Python version:

In [5]:
from classiq import *


@qfunc
def prepare_minus(target: Output[QBit]):
    allocate(out=target, num_qubits=1)
    X(target)
    H(target)


@qfunc
def oracle_function(target: QBit, x: QNum):
    target ^= x == 0


@qfunc
def oracle_phase_kickback(x: QNum):
    target = QBit("target")
    within_apply(
        within=lambda: prepare_minus(target), apply=lambda: oracle_function(target, x)
    )


@qfunc
def main(x: Output[QNum]):
    allocate(num_qubits=4, out=x)
    hadamard_transform(x)
    oracle_phase_kickback(x)


qmod = create_model(main, out_file="phase_kickback")
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/2uG1uFdW5FHb7CTWMfDW57ucXeh?version=0.70.0


## References 

<a id='Childs'>[1]</a>: [Lecture Notes on
Quantum Algorithms (Andrew M. Childs)](https://www.cs.umd.edu/~amchilds/qa/qa.pdf)

<a id='LinLin'>[2]</a>: [Lecture Notes on
Quantum Algorithms for Scientific Computations (Lin Lin)](https://math.berkeley.edu/~linlin/qasc/qasc_notes.pdf)

<a id='Wolf'>[3]</a>: [Quantum Computing:
Lecture Notes (Ronald de Wolf)](https://homepages.cwi.nl/~rdewolf/qcnotes.pdf)

<a id='Nielsen&Chunag'>[4]</a>: [Quantum Computation and Quantum Information (Michael Nielsen and Isaac Chuang)](https://en.wikipedia.org/wiki/Quantum_Computation_and_Quantum_Information)
